In [1]:
import pandas as pd

In [3]:
customers_df = pd.read_csv('customer.csv')

In [10]:
a = 10/3

In [11]:
int(a)

3

In [4]:
customers_df.postcode=customers_df.postcode.mask(customers_df['state'].str.len()+customers_df['gender'].str.len()+customers_df['age'].astype('str').str.len()>5,'******')

In [6]:
customers_df['state'].str.len()

0      3
1      3
2      3
3      2
4      2
      ..
110    2
111    2
112    2
113    2
114    2
Name: state, Length: 115, dtype: int64

In [5]:
customers_df

,person_id,postcode,state,gender,age,account_type,loyal_customer
0,750-67-8428,******,NSW,F,19,credit,True
1,226-31-3081,******,NSW,F,22,credit,True
2,631-41-3108,******,NSW,M,32,credit,True
3,123-19-1176,2000,WA,M,32,credit,True
4,373-73-7910,2000,WA,M,32,credit,True
...,...,...,...,...,...,...,...
110,509-56-0000,822,NT,M,52,credit,True
111,305-64-0000,822,NT,M,52,credit,True
112,202-56-0000,822,NT,M,52,p_loan,True
113,809-34-0000,822,NT,M,52,p_loan,True


In [1]:
# from pyspark.sql import SparkSession
import pyspark.pandas as ps

ModuleNotFoundError: No module named 'pyspark'

In [11]:
# spark = SparkSession.builder.getOrCreate()
ps.set_option('compute.ops_on_diff_frames', True)

In [12]:
# Loading csv data into pyspark DataFrame
customers=spark.read.options(header=True,inferSchema=True).csv('C:\Users\avitr\Downloads\customer.csv')
transactions=spark.read.options(header=True,inferSchema=True).csv('C:\Users\avitr\Downloads\transactions.csv')

In [29]:
# Converting pyspark DataFrame to pandas DataFrame
customers_df=customers.pandas_api()
transactions_df=transactions.pandas_api()

In [251]:
customers_df.head()

,person_id,postcode,state,gender,age,account_type,loyal_customer
0,750-67-8428,2000,NSW,F,19,credit,True
1,226-31-3081,2000,NSW,F,22,credit,True
2,631-41-3108,2000,NSW,M,32,credit,True
3,123-19-1176,2000,WA,M,32,credit,True
4,373-73-7910,2000,WA,M,32,credit,True


In [252]:
transactions_df.head()

,CustomerID,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


If the cell size is larger than 5, hide the customer's postcode for each combination of state, gender, and age

In [275]:
# customers_df[col].str.len() returns the length of the values as a Series
# length of state, gender and age combined should be larger than 5
# data obeying it is obtained using the condition customers_df['state'].str.len()+customers_df['gender'].str.len()+customers_df['age'].astype('str').str.len()>5
# everytime the condition is satisfied, postcode is replaced with ****** using df.mask
# mask replaces a value when the given confition is satisfied
customers_df.postcode=customers_df.postcode.mask(customers_df['state'].str.len()+customers_df['gender'].str.len()+customers_df['age'].astype('str').str.len()>5,'******')

In [276]:
customers_df.head()

,person_id,postcode,state,gender,age,account_type,loyal_customer
0,750-67-8428,******,NSW,F,19,credit,True
1,226-31-3081,******,NSW,F,22,credit,True
2,631-41-3108,******,NSW,M,32,credit,True
3,123-19-1176,2000,WA,M,32,credit,True
4,373-73-7910,2000,WA,M,32,credit,True


Apply bucketing for the age column using the ranges [20-24], [25-29], and [30-34] for 5 years. Ages of 22 and 35, respectively, become [20–24] and [35–39]. Bucketing includes everyone.

In [277]:
# age falls in the range [(age//5)*5,(age//5)*5+4]
customers_df.age='['+((customers_df.age//5)*5).astype('int').astype('str')+'-'+((customers_df.age//5)*5+4).astype('int').astype('str')+']'

In [278]:
customers_df.head()

,person_id,postcode,state,gender,age,account_type,loyal_customer
0,750-67-8428,******,NSW,F,[15-19],credit,True
1,226-31-3081,******,NSW,F,[20-24],credit,True
2,631-41-3108,******,NSW,M,[30-34],credit,True
3,123-19-1176,2000,WA,M,[30-34],credit,True
4,373-73-7910,2000,WA,M,[30-34],credit,True


correct or add loyalty flag (column loyal_customer, values true/false)
\
using following conditions: if customer transacted on the amount greater
\
than 1000 per month loyal_customer = true, otherwise false Note: each 
\
transaction total amount is in column Total

In [257]:
# to access datetime features, convert Date column to datetime object using to_datetime()
dates=ps.to_datetime(transactions_df.Date,format='%m/%d/%Y')

In [279]:
# groupby CustomerID gives all the money spent by that customer
# groupby Year after CustomerID gives all the money spent by that customer in every year
# groupby Month after CustomerID, Year gives all the money spent by that customer in every month of every year
# sum of the Total money spent in a month should be greater than 1000 for all months
# it is for every month using transactions_df.groupby(['CustomerID',dates.dt.year.rename('Year'),dates.dt.month.rename('Month')]).Total.sum()>1000
# to check if the condition is obeyed for all months,
# first apply reset_index() to access the grouped values as a DataFrame
# now the boolen indicating if the spend is greater than 1000 for each month should be True for all months
# it is ensured by first grouping by CustomerID and applying .all()
# a Series of booleans indexed by CustomerID is obtained
loyal_customer=((transactions_df.groupby(['CustomerID',dates.dt.year.rename('Year'),dates.dt.month.rename('Month')]).Total.sum()>1000).reset_index()).groupby('CustomerID').Total.all().rename('loyal_customer')

In [280]:
# customer_df contains loyal_customer but it is wrong
# drop it
# join the customer_df with the loyal_customer values obtained
customers_df=customers_df.drop('loyal_customer',axis=1).join(loyal_customer,on='person_id')

In [281]:
customers_df.head()

,person_id,postcode,state,gender,age,account_type,loyal_customer
0,750-67-8428,******,NSW,F,[15-19],credit,True
1,226-31-3081,******,NSW,F,[20-24],credit,False
2,631-41-3108,******,NSW,M,[30-34],credit,False
3,123-19-1176,2000,WA,M,[30-34],credit,False
4,373-73-7910,2000,WA,M,[30-34],credit,False


filter out transactions of customers that do not present in the customer
dataset after previous treatments.

In [282]:
# right join transactions_df with customers_df.person_id
# this way, CustomerID values not in customers_df are eliminated
# drop the person_id column as it is not required
transactions_df=transactions_df.join(customers_df.person_id,how='right').drop('person_id',axis=1)

In [283]:
transactions_df

,CustomerID,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,529-56-3974,Sports and travel,16.49,2,1.6490,34.6290,2/5/2019,11:32,Ewallet,32.98,4.761905,1.6490,4.6
111,365-64-0515,Health and beauty,98.21,3,14.7315,309.3615,2/5/2019,10:41,Credit card,294.63,4.761905,14.7315,7.8
112,252-56-2699,Fashion accessories,72.84,7,25.4940,535.3740,2/15/2019,12:44,Cash,509.88,4.761905,25.4940,8.4
113,829-34-3910,Home and lifestyle,58.07,9,26.1315,548.7615,1/19/2019,20:07,Ewallet,522.63,4.761905,26.1315,4.3


Provide summary of spendings by days of week: how much customers
spend on Mondays, Tuesdays ...Sundays

In [284]:
# groupby day of the week and describe
transactions_df.groupby([dates.dt.dayofweek]).Total.describe()

,count,mean,std,min,25%,50%,75%,max
Date,,,,,,,,
0,14.0,478.776750,233.177078,71.5260,409.720500,500.40900,626.936625,829.0800
1,18.0,408.494917,264.008075,34.6290,200.400375,383.40225,603.760500,867.6150
2,15.0,240.842000,165.387925,44.5935,127.968750,172.74600,390.253500,535.7205
3,11.0,409.731000,300.608688,33.9360,126.304500,367.03800,712.992000,827.0850
4,20.0,359.627100,288.496713,16.2015,79.647750,313.31475,560.125125,922.6350
5,19.0,310.246816,228.065291,19.2465,141.996750,279.18450,466.179000,931.0350
6,18.0,435.252417,265.125039,70.2870,238.825125,345.81225,633.945375,939.5400


For transactions happening on Wednesdays cap the Total amount to 99
if the amount is more or equal to 100 (what do you do to the Tax, Unit price
and Quantity columns in this case?)

In [285]:
# if total>=100 on wednesdays, total is changed to 99
# wednesday is ensured using dates.dt.day_of_week==2
# total is ensured using transactions_df.Total>=100
# unit_price becomes 99/(1.05*quantity)
transactions_df['Unit price']=transactions_df['Unit price'].mask((dates.dt.day_of_week==2)&(transactions_df.Total>=100),99/(1.05*transactions_df['Quantity']))
# cogs is unit_price*quantity
transactions_df.cogs=transactions_df.cogs.mask((dates.dt.day_of_week==2)&(transactions_df.Total>=100),transactions_df['Quantity']*transactions_df['Unit price'])
# tax is 5% of cogs
transactions_df['Tax 5%']=transactions_df['Tax 5%'].mask((dates.dt.day_of_week==2)&(transactions_df.Total>=100),transactions_df.cogs*0.05)
# gross income is tax
transactions_df['gross income']=transactions_df['Tax 5%'].round(4)
transactions_df.Total=transactions_df.Total.mask((dates.dt.day_of_week==2)&(transactions_df.Total>=100),99)

In [286]:
transactions_df.head(10)

,CustomerID,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,Health and beauty,74.690000,7,26.141500,548.9715,1/5/2019,13:08,Ewallet,522.830000,4.761905,26.1415,9.1
1,226-31-3081,Electronic accessories,15.280000,5,3.820000,80.2200,3/8/2019,10:29,Cash,76.400000,4.761905,3.8200,9.6
2,631-41-3108,Home and lifestyle,46.330000,7,16.215500,340.5255,3/3/2019,13:23,Credit card,324.310000,4.761905,16.2155,7.4
3,123-19-1176,Health and beauty,58.220000,8,23.288000,489.0480,1/27/2019,20:33,Ewallet,465.760000,4.761905,23.2880,8.4
4,373-73-7910,Sports and travel,86.310000,7,30.208500,634.3785,2/8/2019,10:37,Ewallet,604.170000,4.761905,30.2085,5.3
5,699-14-3026,Electronic accessories,85.390000,7,29.886500,627.6165,3/25/2019,18:30,Ewallet,597.730000,4.761905,29.8865,4.1
6,355-53-5943,Electronic accessories,68.840000,6,20.652000,433.6920,2/25/2019,14:36,Ewallet,413.040000,4.761905,20.6520,5.8
7,315-22-5665,Home and lifestyle,73.560000,10,36.780000,772.3800,2/24/2019,11:38,Ewallet,735.600000,4.761905,36.7800,8.0
8,665-32-9167,Health and beauty,36.260000,2,3.626000,76.1460,1/10/2019,17:15,Credit card,72.520000,4.761905,3.6260,7.2
9,692-92-5582,Food and beverages,31.428571,3,4.714286,99.0000,2/20/2019,13:27,Credit card,94.285714,4.761905,4.7143,5.9


For column Time in transactions round down by 15 min periods.
Example: 19:21 - becomes 19:15, 17:36 - becomes 17:30, 10:12 - becomes
10:00, 12:50 - becomes 12:45

In [287]:
# minutes in time can be obtained from 3rd index
# transactions_df.Time.str[3:]
# ((transactions_df.Time.str[3:]).astype('int')//15)*15 gives the rounded down value of minutes
# this is converted to string
# we have 0,15,30,45 as possible values
# but 0 is represented as 00 in minutes
# it is achieved using .replace('0','00')
# rounded down minutes is added hours and assigned to Time
transactions_df.Time=transactions_df.Time.str[:3]+(((transactions_df.Time.str[3:]).astype('int')//15)*15).astype('str').replace('0','00')

In [288]:
transactions_df.head()

,CustomerID,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:00,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:15,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:15,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:30,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:30,Ewallet,604.17,4.761905,30.2085,5.3


In [289]:
# saving data
customers_df=customers_df.to_spark()
customers_df.toPandas().to_csv('customer_final.csv')
transactions_df=transactions_df.to_spark()
transactions_df.toPandas().to_csv('transactions_final.csv')